# Team HexCode: Eagle🦅

In [1]:
import pandas as pd
import numpy as np
import cv2
# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
# Scaler [0 ==> 1]
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
# Load the .npz fil|e
realdata = np.load('real.npz')
fakedata = np.load('fake.npz')

In [3]:
x_real = realdata['x']
y_real = realdata['y']

x_fake = fakedata['x']
y_fake = fakedata['y']

In [4]:
for i in range(len(x_real)):
    x_real[i][np.isinf(x_real[i])] = np.nan


for i in range(len(x_fake)):
    x_fake[i][np.isinf(x_fake[i])] = np.nan

for i in range(len(x_real)):
    nan_indices = np.isnan(x_real[i])
    non_nan_values = x_real[i][~nan_indices]
    random_values = np.random.choice(non_nan_values, size=np.sum(nan_indices))
    x_real[i][nan_indices] = random_values

for i in range(len(x_fake)):
    nan_indices = np.isnan(x_fake[i])
    non_nan_values = x_fake[i][~nan_indices]
    random_values = np.random.choice(non_nan_values, size=np.sum(nan_indices))
    x_fake[i][nan_indices] = random_values

In [5]:
# Reshape x to be a 2D array
x_reshaped_real = x_real.reshape(x_real.shape[0], -1)
x_reshaped_fake = x_fake.reshape(x_fake.shape[0], -1)

In [6]:
# Loading DataSet into a 2d DataFrame

# Real
dfx_real = pd.DataFrame(x_reshaped_real)
dfy_real = pd.DataFrame(y_real)

# Samples from Real
dfx_sample_real = pd.DataFrame(x_real[700])
dfy_sample_real = pd.DataFrame(y_real[700])

# Fake
dfx_fake = pd.DataFrame(x_reshaped_fake)

# Samples from Fake
dfx_sample_fake = pd.DataFrame(x_fake[370])
dfy_sample_fake = pd.DataFrame(y_fake[370])

In [7]:
dfx_real.head()

,0,1,2,3,4,5,6,7,8,9,...,201788,201789,201790,201791,201792,201793,201794,201795,201796,201797
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.111031e-04,0.000040,2.998114e-05,0.000009,0.000025,0.000036,0.000009,0.000004,1.913309e-05,2.330542e-05
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.494190e-05,0.000025,8.344650e-07,0.000009,0.000047,0.000051,0.000007,0.000023,2.205372e-06,2.539158e-05
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.980232e-07,0.000002,1.251698e-05,0.000066,0.000028,0.000008,0.000012,0.000002,1.192093e-07,4.649162e-06
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.270937e-05,0.000003,5.763769e-05,0.000103,0.000081,0.000008,0.000007,0.000010,4.470348e-06,5.960464e-08
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.159046e-04,0.000081,2.744198e-04,0.000032,0.000081,0.000023,0.000003,0.000252,6.842613e-04,3.945827e-04


# Preparing Dataset fake/real

In [8]:
# Create a DataFrame with a column containing 750 1's
real_label = pd.DataFrame({'real_fake': [1] * 750})
train_Data1 = pd.concat([dfx_real, real_label], axis=1)
train_Data1

,0,1,2,3,4,5,6,7,8,9,...,201789,201790,201791,201792,201793,201794,201795,201796,201797,real_fake
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000040,2.998114e-05,0.000009,0.000025,0.000036,0.000009,0.000004,1.913309e-05,2.330542e-05,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000025,8.344650e-07,0.000009,0.000047,0.000051,0.000007,0.000023,2.205372e-06,2.539158e-05,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000002,1.251698e-05,0.000066,0.000028,0.000008,0.000012,0.000002,1.192093e-07,4.649162e-06,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000003,5.763769e-05,0.000103,0.000081,0.000008,0.000007,0.000010,4.470348e-06,5.960464e-08,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000081,2.744198e-04,0.000032,0.000081,0.000023,0.000003,0.000252,6.842613e-04,3.945827e-04,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000006,2.896786e-05,0.000046,0.000030,0.000038,0.000046,0.000016,2.080202e-05,2.855062e-05,1
746,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000003,4.351139e-06,0.000026,0.000019,0.000004,0.000006,0.000006,4.208088e-05,2.145767e-05,1
747,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000022,4.327297e-05,0.000022,0.000024,0.000003,0.000003,0.000007,1.013279e-05,1.913309e-05,1
748,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000039,1.919270e-05,0.000013,0.000005,0.000010,0.000006,0.000010,4.416704e-05,6.669760e-05,1


In [9]:
# Create a DataFrame with a column containing 750 0's
fake_label = pd.DataFrame({'real_fake': [0] * 750})
train_Data0 = pd.concat([dfx_real, fake_label], axis=1)
train_Data0

,0,1,2,3,4,5,6,7,8,9,...,201789,201790,201791,201792,201793,201794,201795,201796,201797,real_fake
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000040,2.998114e-05,0.000009,0.000025,0.000036,0.000009,0.000004,1.913309e-05,2.330542e-05,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000025,8.344650e-07,0.000009,0.000047,0.000051,0.000007,0.000023,2.205372e-06,2.539158e-05,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000002,1.251698e-05,0.000066,0.000028,0.000008,0.000012,0.000002,1.192093e-07,4.649162e-06,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000003,5.763769e-05,0.000103,0.000081,0.000008,0.000007,0.000010,4.470348e-06,5.960464e-08,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000081,2.744198e-04,0.000032,0.000081,0.000023,0.000003,0.000252,6.842613e-04,3.945827e-04,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000006,2.896786e-05,0.000046,0.000030,0.000038,0.000046,0.000016,2.080202e-05,2.855062e-05,0
746,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000003,4.351139e-06,0.000026,0.000019,0.000004,0.000006,0.000006,4.208088e-05,2.145767e-05,0
747,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000022,4.327297e-05,0.000022,0.000024,0.000003,0.000003,0.000007,1.013279e-05,1.913309e-05,0
748,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000039,1.919270e-05,0.000013,0.000005,0.000010,0.000006,0.000010,4.416704e-05,6.669760e-05,0


In [10]:
Train_Data = pd.concat([train_Data1, train_Data0], axis=0)
Train_Data=Train_Data.reset_index()
Train_Data.drop(columns=['index'],axis=1,inplace=True)
Train_Data

,0,1,2,3,4,5,6,7,8,9,...,201789,201790,201791,201792,201793,201794,201795,201796,201797,real_fake
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000040,2.998114e-05,0.000009,0.000025,0.000036,0.000009,0.000004,1.913309e-05,2.330542e-05,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000025,8.344650e-07,0.000009,0.000047,0.000051,0.000007,0.000023,2.205372e-06,2.539158e-05,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000002,1.251698e-05,0.000066,0.000028,0.000008,0.000012,0.000002,1.192093e-07,4.649162e-06,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000003,5.763769e-05,0.000103,0.000081,0.000008,0.000007,0.000010,4.470348e-06,5.960464e-08,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000081,2.744198e-04,0.000032,0.000081,0.000023,0.000003,0.000252,6.842613e-04,3.945827e-04,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000006,2.896786e-05,0.000046,0.000030,0.000038,0.000046,0.000016,2.080202e-05,2.855062e-05,0
1496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000003,4.351139e-06,0.000026,0.000019,0.000004,0.000006,0.000006,4.208088e-05,2.145767e-05,0
1497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000022,4.327297e-05,0.000022,0.000024,0.000003,0.000003,0.000007,1.013279e-05,1.913309e-05,0
1498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000039,1.919270e-05,0.000013,0.000005,0.000010,0.000006,0.000010,4.416704e-05,6.669760e-05,0


In [11]:
data_2d = x_real.reshape(-1, x_real.shape[1], x_real.shape[2])

data_list = [{'samples': sample} for sample in data_2d]

df = pd.DataFrame(data_list)

In [12]:
data_2d = x_fake.reshape(-1, x_fake.shape[1], x_fake.shape[2])

data_list = [{'samples': sample} for sample in data_2d]

df_fake = pd.DataFrame(data_list)

In [13]:
df = pd.concat([df,real_label],axis=1)
df

,samples,real_fake
0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",1
1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",1
2,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",1
3,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",1
4,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",1
...,...,...
745,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",1
746,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",1
747,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",1
748,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",1


In [14]:
df_fake = pd.concat([df_fake,fake_label],axis=1)
df_fake

,samples,real_fake
0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0
1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0
2,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0
3,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0
4,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0
...,...,...
745,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0
746,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0
747,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0
748,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0


In [15]:
Train_Data = pd.concat([df, df_fake], axis=0)
Train_Data=Train_Data.reset_index()
Train_Data.drop(columns=['index'],axis=1,inplace=True)
Train_Data

,samples,real_fake
0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",1
1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",1
2,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",1
3,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",1
4,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",1
...,...,...
1495,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0
1496,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0
1497,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0
1498,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0


In [16]:
Train_Data['samples'][2].min()

0.0

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import librosa.display
import pylab
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import librosa
import glob
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras import optimizers
import keras

In [ ]:
Train_Data['samples'][3].min()

0.0

In [ ]:
Train_Data = Train_Data.sample(frac=1).reset_index(drop=True)


In [ ]:
scaler = StandardScaler()

# Apply the scaler to each cell in the DataFrame
Train_Data['samples'] = Train_Data['samples'].apply(lambda x: scaler.fit_transform(x))

In [ ]:
Train_Data

,samples,real_fake
0,"[[-0.1033, -0.1301, -0.1364, -0.156, -0.1667, ...",0
1,"[[-0.1575, -0.2024, -0.2051, -0.1598, -0.1687,...",0
2,"[[-0.0784, -0.1162, -0.11597, -0.1449, -0.1481...",1
3,"[[-0.2098, -0.2239, -0.1937, -0.1802, -0.1769,...",1
4,"[[-0.2297, -0.2235, -0.199, -0.1526, -0.1769, ...",1
...,...,...
1495,"[[-0.1445, -0.1727, -0.1458, -0.1451, -0.1481,...",0
1496,"[[-0.1009, -0.145, -0.1307, -0.1382, -0.1665, ...",1
1497,"[[-0.2275, -0.238, -0.1853, -0.2075, -0.2517, ...",1
1498,"[[-0.1526, -0.3445, -0.3464, -0.2817, -0.2441,...",1


In [ ]:
X = np.array(Train_Data['samples'].tolist())

In [ ]:
X.shape

(1500, 1998, 101)

In [ ]:
X = np.expand_dims(X, axis=-1)

In [ ]:
y= to_categorical(np.array(Train_Data['real_fake']),num_classes=2)
# y = np.expand_dims(y, axis=-1)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state = 42)

In [ ]:
X_train

In [ ]:
y.shape

(1500, 2)

In [ ]:
X_test.shape

(300, 1998, 101, 1)

In [ ]:
from keras import regularizers

from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense, Activation, BatchNormalization
# Define the model
model = Sequential()

# First Convolution Block with Relu and Batch Norm
model.add(Conv2D(8, (5, 5), strides=(2, 2), padding='same', kernel_regularizer=regularizers.l2(0.01)))
model.add(Activation('relu'))
model.add(BatchNormalization())

# Second Convolution Block
model.add(Conv2D(16, (3, 3), strides=(2, 2), padding='same', kernel_regularizer=regularizers.l2(0.01)))
model.add(Activation('relu'))
model.add(BatchNormalization())

# Third Convolution Block
model.add(Conv2D(32, (3, 3), strides=(2, 2), padding='same', kernel_regularizer=regularizers.l2(0.01)))
model.add(Activation('relu'))
model.add(BatchNormalization())

# Fourth Convolution Block
model.add(Conv2D(64, (3, 3), strides=(2, 2), padding='same', kernel_regularizer=regularizers.l2(0.01)))
model.add(Activation('relu'))
model.add(BatchNormalization())


model.add(Conv2D((64+64), (3, 3), strides=(2, 2), padding='same', kernel_regularizer=regularizers.l2(0.01)))
model.add(Activation('relu'))
model.add(BatchNormalization())
# Adaptive pool and flatten for input to linear layer
model.add(Flatten())
model.add(Dense(64, kernel_regularizer=regularizers.l2(0.01)))
# Linear layer
model.add(Dense(2, kernel_regularizer=regularizers.l2(0.01)))
model.add(Activation('softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [ ]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau
early_stopping = EarlyStopping(patience=10, min_delta=0.001,restore_best_weights=True)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience=3, verbose=1,
                                            factor=0.5, min_lr=0.00001)

In [ ]:
epochs = 200
batch_size = 64

history = model.fit(X_train, y_train, batch_size=batch_size,
                              epochs = epochs, validation_data = (X_test,y_test), steps_per_epoch= X_train.shape[0] // batch_size, callbacks= [learning_rate_reduction])

Epoch 1/200
18/18 [==============================] - 76s 4s/step - loss: 5.9201 - accuracy: 0.8108 - val_loss: 4.1061 - val_accuracy: 0.5167 - lr: 0.0010
Epoch 2/200
18/18 [==============================] - 72s 4s/step - loss: 3.7379 - accuracy: 0.9560 - val_loss: 4.7939 - val_accuracy: 0.5167 - lr: 0.0010
Epoch 3/200
18/18 [==============================] - 71s 4s/step - loss: 3.0548 - accuracy: 0.9842 - val_loss: 5.7978 - val_accuracy: 0.5167 - lr: 0.0010
Epoch 4/200
18/18 [==============================] - ETA: 0s - loss: 2.9417 - accuracy: 0.9798
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
18/18 [==============================] - 66s 4s/step - loss: 2.9417 - accuracy: 0.9798 - val_loss: 9.2289 - val_accuracy: 0.5167 - lr: 0.0010
Epoch 5/200
18/18 [==============================] - 68s 4s/step - loss: 2.6306 - accuracy: 0.9930 - val_loss: 10.3030 - val_accuracy: 0.5167 - lr: 5.0000e-04
Epoch 6/200
18/18 [==============================] - 67s 4s/step -

KeyboardInterrupt: 

In [ ]:

loss, acc = model.evaluate(X_test, y_test, verbose=0)
print('Test loss: {}'.format(loss))
print('Test Accuracy: {}'.format(acc))

In [ ]:
model.save('model.h5')

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
from keras.models import load_model
import numpy as np

model = load_model('/content/model.h5')
inputs = [x_real[75], x_fake[10],x_real[749]]
probabilities=[]
for input in inputs:
  input[np.isinf(input)] = np.nan
  nan_indices = np.isnan(input)
  non_nan_values = input[~nan_indices]
  random_values = np.random.choice(non_nan_values, size=np.sum(nan_indices))
  input[nan_indices] = random_values

  scaler = StandardScaler()

  input = scaler.fit_transform(input)
  input = np.expand_dims(input, axis=-1)
  print(input.shape)
  input = np.expand_dims(input, axis=(0, 3))
  print(input.shape)
  probabilities.append(model.predict(input))




(1998, 101, 1)
(1, 1998, 101, 1, 1)
1/1 [==============================] - 0s 440ms/step
(1998, 101, 1)
(1, 1998, 101, 1, 1)
1/1 [==============================] - 0s 65ms/step
(1998, 101, 1)
(1, 1998, 101, 1, 1)
1/1 [==============================] - 0s 85ms/step


In [ ]:
from keras.models import load_model
import numpy as np
scaler = StandardScaler()
model = load_model('/content/model.h5')
inputs = [x_real[75], x_fake[10],x_real[749]]

min_value = float('inf')
reals_index=[]
Reals=[]
for input in inputs:
    input[np.isinf(input)] = np.nan
    nan_indices = np.isnan(input)
    non_nan_values = input[~nan_indices]
    random_values = np.random.choice(non_nan_values, size=np.sum(nan_indices))
    input[nan_indices] = random_values
    input = scaler.fit_transform(input)
    input = np.expand_dims(input, axis=-1)
    input = np.expand_dims(input, axis=(0, 3))
    prediction = model.predict(input)[0][1]
    if (prediction) == 1:
        print("yes")
        Reals.append(prediction)
        reals_index.append(inputs.index(input))
if len(Reals)==1:
    print(f"reals_index[0]+1{reals_index[0]+1}")
elif len(Reals)>1:
    for real in Reals:
        if real[0][0]<min_value:
            min_value = real[0][0]
            index = reals_index[Reals.index(real)]
    print(f"index+1 {index+1}")

else:
    print("None")

ValueError: Name tf.RaggedTensorSpec has already been registered for class tensorflow.python.ops.ragged.ragged_tensor.RaggedTensorSpec.

In [ ]:
"from sklearn.metrics import classification_report
import numpy as np

# Assuming 'y_test' is a binary indicator (one-hot encoded)
y_true_labels = np.argmax(y_test, axis=1)

# Assuming 'y_pred_classes' is your predicted classes
print(classification_report(y_true_labels, y_pred_classes))

In [ ]:
for idx, probability in enumerate(probabilities):
    if probability[0][1] == 1:
        print(idx)

In [ ]:
probabilities

[]

In [ ]:
probabilities
# fake,real

array([[1., 0.]], dtype=float32)

In [ ]:
x_real[0]

array([[0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       ...,
       [3.359e-04, 3.683e-02, 7.349e-02, ..., 5.215e-05, 8.821e-05,
        6.813e-05],
       [6.535e-03, 9.628e-03, 2.792e-02, ..., 1.311e-06, 2.420e-05,
        2.646e-05],
       [2.278e-02, 1.903e-02, 5.807e-02, ..., 4.053e-06, 1.913e-05,
        2.331e-05]], dtype=float16)